In [47]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.workflow import Context
from llama_index.core.tools import FunctionTool
import asyncio
import os

In [33]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(
    model="qwen3:0.6b",
    request_timeout=360.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [34]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
cart = []

async def search_furniture_products(query: str) -> str:
    """
    Tool: search_furniture_products

    Use this function when the user asks a natural language question about 
    the products, inventory, or details of furniture store items 
    (e.g., sofas, tables, chairs, prices, materials, or availability).

    The function queries the product database or document index to find 
    the most relevant information about the requested furniture item.

    Input:
        query (str): The natural language question or keyword about furniture items.

    Output:
        str: A summarized natural language answer containing relevant 
        product details and information.
    """
    response = await query_engine.aquery(query)
    return str(response)

def addToCart(item)->str:
    """Useful for adding an item to the cart"""
    cart.append(item)
    return "Item added to the cart"

# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[addToCart,search_furniture_products],
    llm=Ollama(model="qwen3:0.6b",thinking=False),
    system_prompt="""You are a helpful assistant that can : 
    1. Add an item to the cart.
    2.Search and give information about a furniture store item, use search_furniture_products
    """,
)

In [59]:
ctx = Context(agent)

In [50]:
respnse = await agent.run("Add trimmer to the cart")

In [51]:
respnse

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Item added to the cart.')]), structured_response=None, current_agent_name='Agent', raw={'model': 'qwen3:0.6b', 'created_at': '2025-10-22T05:51:08.690907173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 389452081, 'load_duration': 49095859, 'prompt_eval_count': 452, 'prompt_eval_duration': 236327070, 'eval_count': 7, 'eval_duration': 87262190, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None), 'usage': {'prompt_tokens': 452, 'completion_tokens': 7, 'total_tokens': 459}}, tool_calls=[ToolCallResult(tool_name='addToCart', tool_kwargs={'item': 'trimmer'}, tool_id='addToCart', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='Item added to the cart')], tool_name='addToCart', raw_input={'args': (), 'kwargs': {'item': 'trimmer'}}, raw_output='Item adde

In [52]:
cart

['trimmer']

In [84]:
response = await agent.run("What are the products available in your furniture store")

In [85]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text="I don't have a furniture store, but I can help you search for furniture products. Could you provide a query or keyword about furniture items you're interested in?")])

In [86]:
response = await agent.run("What dining tables you have in your furniture store")

In [87]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text="I'm sorry, but I can't access real-time information about the furniture store. Could you please provide more details or clarify your question?")])

In [71]:
response = await agent.run("What products do you have in your furniture store?")

In [72]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='None of the tools provided can be used to answer the question about the products available in the furniture store.')])

In [80]:
response = await query_engine.aquery("what products do you have in your furniture store")
resp = str(response)

In [81]:
print(resp)

<think>
Okay, the user is asking, "what products do you have in your furniture store?" and they provided a list of products in the Details.txt file. Let me check the context to make sure I don't include any prior knowledge.

Looking at the context, there are five entries in the file: FUR001, FUR002, FUR003, FUR004, and FUR005. Each entry has details like category, price, etc. The user wants to know the products available. Since the context is the list provided, I need to list them all without adding any information I don't have. The answer should just be the product names in the list. Let me make sure there's no mention of other products or categories beyond the ones listed. Yep, all five are in the list. So the answer is the names of each product as listed.
</think>

The furniture store has the following products:  
- Modern Wooden Dining Table  
- ErgoComfort Office Chair  
- Velvet Queen Size Bed  
- Glass Top Coffee Table  
- Bookshelf with Cabinet


In [82]:
response = await agent.achat("what products do you have in your furniture store?")

AttributeError: 'FunctionAgent' object has no attribute 'achat'